# Imports
To begin, import these:

In [1]:
%load_ext autoreload

In [8]:
# To reload: 
%autoreload

In [3]:
from parser import *
from trainer import *

Folders where files are stored:

In [4]:
# inputXML = "sample_train"
# testXMLFolder = "sample_test"
inputXML = "sample_train_1"
testXMLFolder = "sample_test_1"
summaryFolder = "sample_summary"
bodyFolder = "sample_body"

# Parse
Here is where we read and parse all of the Wikipedia articles.

In [9]:
documents, labels = parse_xml_folder(inputXML, summaryFolder, bodyFolder, writeOption='none')
surfaceFeatures = process_data(documents)

<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
STAGE [1] -- PARSING XML -- from sample_train_1 ...


UnicodeEncodeError: 'ascii' codec can't encode character u'\u2013' in position 90: ordinal not in range(128)

# Train
After parsing, take the documents & labels and then train our model.

In [5]:
features = featurize(documents, surfaceFeatures)
model, featMatrix, vectorizer = train_classifier(features, labels)

scores = cross_val_score(model, featMatrix, labels, scoring="f1_macro") # accuracy, f1, log_loss
print model.coef_
predictions = model.predict(featMatrix)
print metrics.classification_report(labels, predictions)

STAGE [3] -- FEATURIZING -- (TextRank, LexRank, LDA) ...
STAGE [4] -- TRAINING MODEL -- Logistic Regression ...
<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
[[-0.06978593 -0.15334872 -0.25510274  0.0126926   0.1922343  -0.24831293
  -0.02436139  0.68096096  0.79596326 -0.22309355 -0.43292563  0.05515176
  -0.61671967  0.36719587 -0.24619082  0.022105    0.26400959 -0.03795395
   0.06248649 -0.43588031  0.1884946   0.60676931 -0.11005708 -0.28079814]]
             precision    recall  f1-score   support

       BODY       0.96      0.72      0.82       151
    SUMMARY       0.16      0.67      0.25        12

avg / total       0.90      0.71      0.78       163



# Test
Then, we test on the test data from the testXML folder

In [6]:
testDocuments, testLabels = parse_xml_folder(testXMLFolder, summaryFolder, bodyFolder, writeOption='none')
testSurfaceFeatures = process_data(testDocuments)

<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
STAGE [1] -- PARSING XML -- from sample_test_1 ...
STAGE [2] -- PROCESSING DATA -- (tokenizing/tagging/stopwords/extracting) ...


In [7]:
testFeatures = featurize(testDocuments, testSurfaceFeatures)
testFeatureMatrix = vectorizer.transform(testFeatures) # Features = List of counters
print evaluate_trained_classifier(model, testFeatureMatrix, testLabels)

STAGE [3] -- FEATURIZING -- (TextRank, LexRank, LDA) ...
STAGE [5] -- TESTING -- Logistic Regression ...
<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
[ 0.43861184  0.41860465  0.34155518]
             precision    recall  f1-score   support

       BODY       0.95      0.34      0.50       217
    SUMMARY       0.04      0.60      0.08        10

avg / total       0.91      0.35      0.48       227



# ROUGE Analysis 
To evaluate against ROUGE, we first construct a summary for each document.

In [8]:
def extract_summary(document):
    summary = []
    for su in document:
        if su.label == 'summary':
            summary.append(su.text)
        else:
            break
    return summary

In [9]:
from heapq import nlargest

def extract_top_ranked(document, featureMatrix, num):
    predictions = model.predict_proba(featureMatrix)
    itr = range(len(document))
    topIndexes = nlargest(num, itr, key=lambda i: predictions[i][1])
    topSentences = [ document[index].text for index in topIndexes]
    return topSentences

# Run ROUGE
Now, we run ROUGE against the gold summaries.

In [10]:
featIndex = 0
rouge_gold_summaries = []
rouge_generated_summaries = []
for i, tdoc in enumerate(testDocuments):
    numSentences = len(tdoc)
    rouge_gold_summaries.append(extract_summary(tdoc))
    rouge_generated_summaries.append(extract_top_ranked(tdoc, testFeatureMatrix[featIndex:featIndex+numSentences], 10))
    featIndex += numSentences

#print rouge_gold_summaries
#print rouge_generated_summaries

In [11]:
pickle_gold_summaries = {}
pickle_generated_summaries = {}
for i in range(len(rouge_gold_summaries)):
    pickle_gold_summaries[i] = rouge_gold_summaries[i]
    pickle_generated_summaries[i] = rouge_generated_summaries[i]

In [12]:
import pickle
pickle.dump( pickle_gold_summaries, open( "pickle_gold_summaries.p", "wb" ) )
pickle.dump( pickle_generated_summaries, open( "pickle_generated_summaries.p", "wb" ) )

In [13]:
%autoreload
from RougeRunner import *

rougeResults = []

loaded_gold_summaries = pickle.load(open( "pickle_gold_summaries.p", "rb" ) )
loaded_generated_summaries = pickle.load( open( "pickle_generated_summaries.p", "rb" ) )

for summaryIndex in range(len(loaded_gold_summaries)):
    gold = loaded_gold_summaries[summaryIndex]
    genSum = loaded_generated_summaries[summaryIndex]
    rougeResults.append(compareUsingRouge(gold, genSum))
    
for result in rougeResults:
    print result

0.06658
